## 학습목표

- Tensorflow V2의 개요와 특징을 파악한다.
- Tensorflow V2의 3가지 주요 API 구성 방식을 이해하고 활용할 수 있다.
- GradientTape를 활용해 보고 좀 더 로우 레벨의 딥러닝 구현 방식을 이해한다

## TensorFlow2 API로 모델 구성하기

### 1) TensorFlow2 Sequential Model

> input 1개, output 1개를 전제로 함

### 2) TensorFlow2 Functional API
> - `keras.Model`을 사용해서 모델을 만든다는 것이 특징이다.  
> - 어떻게 보면 Sequential Model은 그냥 `keras.Model` 클래스를 상속바당서 쓰는 것이다.
> - 입력과 출력을 규정해서 모델을 설계한다.

### 3) TensorFlow2 Subclassing

> - `keras.Model`은 `__init__()`이라는 메서드 안에서 레이어 구성을 정의
> - `call()` 메소드에서 `forward propagation`을 구현

## Tensorflow2 API로 모델 작성하기: MNIST (1) Sequential API 활용

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [10]:
# 데이터 구성부분
mnist = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0

x_train=x_train[...,np.newaxis]
x_test=x_test[...,np.newaxis]

print(len(x_train), len(x_test))

60000 10000


In [11]:
y_train.shape

(60000,)

In [12]:
np.unique(y_train, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8),
 array([5923, 6742, 5958, 6131, 5842, 5421, 5918, 6265, 5851, 5949]))

In [6]:
x_train.shape

(60000, 28, 28)

In [9]:
x_train[...].shape

(60000, 28, 28)

In [16]:
# Sequential Model을 구성해주세요.
"""
Spec:
1. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
2. 64개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
3. Flatten 레이어
4. 128개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
5. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)
"""

# 여기에 모델을 구성해주세요

model = keras.Sequential()
model.add(keras.layers.Conv2D(32, kernel_size=3, activation='relu'))
model.add(keras.layers.Conv2D(64, kernel_size=3, activation='relu'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [19]:
# 모델 학습 설정

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 17s 7ms/step - loss: 0.2043 - accuracy: 0.9381
Epoch 2/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0325 - accuracy: 0.9901
Epoch 3/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0182 - accuracy: 0.9938
Epoch 4/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0095 - accuracy: 0.9969
Epoch 5/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0095 - accuracy: 0.9967
313/313 - 2s - loss: 0.0492 - accuracy: 0.9884


[0.04915982857346535, 0.9883999824523926]

In [20]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
flatten_1 (Flatten)          (None, 36864)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               4718720   
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
Total params: 4,738,826
Trainable params: 4,738,826
Non-trainable params: 0
_________________________________________________________________


## Tensorflow2 API로 모델 작성하기: MNIST (2) Functional API 활용

In [21]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [22]:
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train=x_train[...,np.newaxis]
x_test=x_test[...,np.newaxis]

print(len(x_train), len(x_test))

60000 10000


In [23]:
x_train.shape

(60000, 28, 28, 1)

In [27]:
np.arange(16).reshape(4,2,2)[:,np.newaxis,...].shape

(4, 1, 2, 2)

In [28]:
"""
Spec:
0. (28X28X1) 차원으로 정의된 Input
1. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
2. 64개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
3. Flatten 레이어
4. 128개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
5. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)
"""

# 여기에 모델을 구성해 주세요.

# --- Input
inputs = keras.layers.Input((28, 28, 1))

# --- Conv2D
conv2d = keras.layers.Conv2D(32, kernel_size=3, activation='relu')
x = conv2d(inputs)

# --- Conv2D_1
conv2d_1 = keras.layers.Conv2D(64, kernel_size=3, activation='relu')
x = conv2d_1(x)

# --- Flatten
flatten = keras.layers.Flatten()
x = flatten(x)

# --- Dense
dense = keras.layers.Dense(128, activation='relu')
x = dense(x)

# --- Dense_1
dense_1 = keras.layers.Dense(10, activation='softmax')
outputs = dense_1(x)

# 모델 선언
model = keras.Model(inputs=inputs, outputs=outputs)

```python
# AIFFEL code
inputs = keras.Input(shape=(28, 28, 1))

x = keras.layers.Conv2D(32, 3, activation='relu')(inputs)
x = keras.layers.Conv2D(64, 3, activation='relu')(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128, activation='relu')(x)
predictions = keras.layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=predictions)
```

In [29]:
# 모델 학습 설정

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.2136 - accuracy: 0.9324
Epoch 2/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0332 - accuracy: 0.9899
Epoch 3/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0170 - accuracy: 0.9945
Epoch 4/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0116 - accuracy: 0.9958
Epoch 5/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0079 - accuracy: 0.9975
313/313 - 1s - loss: 0.0415 - accuracy: 0.9903


[0.04145710915327072, 0.9902999997138977]

## Tensorflow2 API로 모델 작성하기: MNIST (3) Subclassing 활용

In [30]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [31]:
# 데이터 구성부분
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train=x_train[...,np.newaxis]
x_test=x_test[...,np.newaxis]

print(len(x_train), len(x_test))

60000 10000


In [58]:
# Subclassing을 활용한 Model을 구성해주세요.
"""
Spec:
0. keras.Model 을 상속받았으며, __init__()와 call() 메서드를 가진 모델 클래스
1. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
2. 64개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
3. Flatten 레이어
4. 128개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
5. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)
6. call의 입력값이 모델의 Input, call의 리턴값이 모델의 Output
"""

# 여기에 모델을 구성해주세요
class CustomModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(32, 3, activation='relu')
        self.conv2 = keras.layers.Conv2D(64, 3, activation='relu')
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(128, activation='relu')
        self.fc2 = keras.layers.Dense(10, activation='softmax')
        
    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        
        return x
        
model = CustomModel()

------------------------------------------------

```python
# DJ practice
class MyModel(keras.Model):
    def __init__(self, input_shape=(28,28)):
        # 여기서 super.__init__() 필요하고 
        self.input_layer = keras.layers.Input(input_shape=input_shape)  # Input 레이어는 써있어도 한번도 call되지 않는단다. 그래서 오류난다.
        self.conv2d = keras.layers.Conv2D(32, kernel_size=3, activation='relu')
        self.conv2d_1 = keras.layers.Conv2D(64, kernel_size=3, activation='relu')
        self.flatten = keras.layers.Flatten()
        self.dense = keras.layers.Dense(128, activation='relu')
        self.dense_1 = keras.layers.Dense(10, activation='softmax')
    
    def call(self):
        inputs = self.input_layer # Input 삭제되고, call(self, inputs)로 들어가야 한다.
        x = self.conv2d(inputs)
        x = self.conv2d_1(x)
        x = self.flatten(x)
        x = self.dense(x)
        outputs = self.dense_1(x)
        
        return keras.model(inputs=inputs, outputs=outputs) # 여기는 그냥 outputs
```

In [54]:
mymodel = MyModel()

In [57]:
# MyModel 클래스 살짝 고쳐서 해봤는데 됐다.
mymodel.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

mymodel.fit(x_train,y_train, epochs=5)
mymodel.evaluate(x_test, y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.2174 - accuracy: 0.9326
Epoch 2/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0375 - accuracy: 0.9886
Epoch 3/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0178 - accuracy: 0.9941
Epoch 4/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0122 - accuracy: 0.9960
Epoch 5/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0074 - accuracy: 0.9980
313/313 - 1s - loss: 0.0416 - accuracy: 0.9896


[0.04158053919672966, 0.9896000027656555]

------------------------------------------------

In [59]:
# 모델 학습 설정

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.2183 - accuracy: 0.9326
Epoch 2/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0333 - accuracy: 0.9897
Epoch 3/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0164 - accuracy: 0.9947
Epoch 4/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0119 - accuracy: 0.9961
Epoch 5/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0096 - accuracy: 0.9967
313/313 - 1s - loss: 0.0485 - accuracy: 0.9886


[0.048538122326135635, 0.9886000156402588]

## TensorFlow2 API로 모델 작성 및 학습하기: CIFAR-100 (1) Sequential API 활용

In [60]:
import tensorflow as tf
from tensorflow import keras

In [62]:
# 데이터 구성부분
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

50000 10000


In [63]:
np.unique(y_train, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
        68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
        85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]),
 array([500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500,
        500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500,
        500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500,
        500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500,
        500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500,
        500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500,
        500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500,
        500, 500, 500, 500, 500, 500, 5

In [64]:
# Sequential Model을 구성해주세요.
"""
Spec:
1. 16개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
2. pool_size가 2인 MaxPool 레이어
3. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
4. pool_size가 2인 MaxPool 레이어
5. 256개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
6. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)
"""

# 여기에 모델을 구성해주세요



AttributeError: module 'tensorflow.keras.layers' has no attribute 'MaxPooling'

```python
model = keras.Sequential()
model.add(keras.layers.Conv2D(16, kernel_size=3, activation='relu'))
model.add(keras.layers.MaxPooling(2)) #<--- MaxPool2D((2,2)) 란다
model.add(keras.layers.Conv2D(32, kernel_size=3, activation='relu'))
model.add(keras.layers.MaxPooling(2))
# 여기도 Flatten()이 들어간다. 안 들어가면 어떻게 될까..?
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dense(100, activation='softmax'))
```

**1) Flatten이 없을 때 어떻게 되는지 실험**  
**2) Flatten 있는데 마지막 노드 갯수를 class 수랑 안 맞추면 어떻게 되는지 실험**

In [66]:
model_noflat = keras.Sequential()
model_noflat.add(keras.layers.Conv2D(16, kernel_size=3, activation='relu'))
model_noflat.add(keras.layers.MaxPool2D((2,2)))
model_noflat.add(keras.layers.Conv2D(32, kernel_size=3, activation='relu'))
model_noflat.add(keras.layers.MaxPool2D((2,2)))
model_noflat.add(keras.layers.Dense(256, activation='relu'))
model_noflat.add(keras.layers.Dense(100, activation='softmax'))

In [67]:
model_noflat.compile(optimizer='adam',
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

model_noflat.fit(x_train, y_train, epochs=5)

model.evaluate(x_test, y_test, verbose=2)

Epoch 1/5


InvalidArgumentError:  logits and labels must have the same first dimension, got logits shape [1152,100] and labels shape [32]
	 [[node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at <ipython-input-67-5ee36d46b7dd>:5) ]] [Op:__inference_train_function_82044]

Function call stack:
train_function


#### # 그럼 flatten 추가해보자

In [68]:
model_noflat = keras.Sequential()
model_noflat.add(keras.layers.Conv2D(16, kernel_size=3, activation='relu'))
model_noflat.add(keras.layers.MaxPool2D((2,2)))
model_noflat.add(keras.layers.Conv2D(32, kernel_size=3, activation='relu'))
model_noflat.add(keras.layers.MaxPool2D((2,2)))
model_noflat.add(keras.layers.Flatten()) # 추가됨
model_noflat.add(keras.layers.Dense(256, activation='relu'))
model_noflat.add(keras.layers.Dense(100, activation='softmax'))

In [70]:
model_noflat.compile(optimizer='adam',
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

model_noflat.fit(x_train, y_train, epochs=2)

model_noflat.evaluate(x_test, y_test, verbose=2)

Epoch 1/2
1563/1563 [==============================] - 5s 3ms/step - loss: 2.1194 - accuracy: 0.4491
Epoch 2/2
1563/1563 [==============================] - 5s 3ms/step - loss: 1.9885 - accuracy: 0.4746
313/313 - 1s - loss: 2.5948 - accuracy: 0.3636


[2.594791889190674, 0.3635999858379364]

#### # softmax 전에 클래스 수와 다른 output 개수를 설정할 경우

In [71]:
model_wrong = keras.Sequential()
model_wrong.add(keras.layers.Conv2D(16, kernel_size=3, activation='relu'))
model_wrong.add(keras.layers.MaxPool2D((2,2)))
model_wrong.add(keras.layers.Conv2D(32, kernel_size=3, activation='relu'))
model_wrong.add(keras.layers.MaxPool2D((2,2)))
model_wrong.add(keras.layers.Flatten()) # 추가됨
model_wrong.add(keras.layers.Dense(256, activation='relu'))
model_wrong.add(keras.layers.Dense(200, activation='softmax'))

In [73]:
model_wrong.compile(optimizer='adam',
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

model_wrong.fit(x_train, y_train, epochs=10)

model_wrong.evaluate(x_test, y_test, verbose=2)

Epoch 1/10
1563/1563 [==============================] - 5s 3ms/step - loss: 2.7935 - accuracy: 0.3053
Epoch 2/10
1563/1563 [==============================] - 5s 3ms/step - loss: 2.5773 - accuracy: 0.3495
Epoch 3/10
1563/1563 [==============================] - 5s 3ms/step - loss: 2.4189 - accuracy: 0.3822
Epoch 4/10
1563/1563 [==============================] - 5s 3ms/step - loss: 2.2619 - accuracy: 0.4168
Epoch 5/10
1563/1563 [==============================] - 5s 3ms/step - loss: 2.1342 - accuracy: 0.4407
Epoch 6/10
1563/1563 [==============================] - 5s 3ms/step - loss: 2.0076 - accuracy: 0.4710
Epoch 7/10
1563/1563 [==============================] - 5s 3ms/step - loss: 1.9053 - accuracy: 0.4953
Epoch 8/10
1563/1563 [==============================] - 5s 3ms/step - loss: 1.8163 - accuracy: 0.5146
Epoch 9/10
1563/1563 [==============================] - 5s 3ms/step - loss: 1.7275 - accuracy: 0.5341
Epoch 10/10
1563/1563 [==============================] - 5s 3ms/step - loss: 1.644

[2.7971274852752686, 0.3531000018119812]

> 아예 작동 안 할 줄 알았는데 작동하긴 한다. 손실함수 계산할 때 y랑 y의 갯수가 다를텐데...  
> broadcasting 된건가... 싶다.  
> 만약 그런 거라면 제대로 된 모델보다 학습이 느리다고만 결론지으면 될 것 같다.

In [74]:
model = keras.Sequential([
    keras.layers.Conv2D(16, 3, activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Conv2D(32, 3, activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(100, activation='softmax')
])

In [75]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 5s 3ms/step - loss: 3.9830 - accuracy: 0.0987
Epoch 2/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.9619 - accuracy: 0.2733
Epoch 3/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.6432 - accuracy: 0.3343
Epoch 4/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.4319 - accuracy: 0.3802
Epoch 5/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.2556 - accuracy: 0.4149
313/313 - 1s - loss: 2.6405 - accuracy: 0.3466


[2.640547037124634, 0.3465999960899353]

> 음... 비슷하다... 뭐지...? 

#### # output 개수 완전 틀리게 해보기

In [76]:
model_wrong = keras.Sequential()
model_wrong.add(keras.layers.Conv2D(16, kernel_size=3, activation='relu'))
model_wrong.add(keras.layers.MaxPool2D((2,2)))
model_wrong.add(keras.layers.Conv2D(32, kernel_size=3, activation='relu'))
model_wrong.add(keras.layers.MaxPool2D((2,2)))
model_wrong.add(keras.layers.Flatten()) # 추가됨
model_wrong.add(keras.layers.Dense(256, activation='relu'))
model_wrong.add(keras.layers.Dense(1000, activation='softmax')) # 100개인데 1000개로

model_wrong.compile(optimizer='adam',
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

model_wrong.fit(x_train, y_train, epochs=10)

model_wrong.evaluate(x_test, y_test, verbose=2)

Epoch 1/10
1563/1563 [==============================] - 5s 3ms/step - loss: 4.2012 - accuracy: 0.0867
Epoch 2/10
1563/1563 [==============================] - 5s 3ms/step - loss: 3.1275 - accuracy: 0.2364
Epoch 3/10
1563/1563 [==============================] - 5s 3ms/step - loss: 2.7714 - accuracy: 0.3043
Epoch 4/10
1563/1563 [==============================] - 5s 3ms/step - loss: 2.5556 - accuracy: 0.3521
Epoch 5/10
1563/1563 [==============================] - 5s 3ms/step - loss: 2.3940 - accuracy: 0.3828
Epoch 6/10
1563/1563 [==============================] - 5s 3ms/step - loss: 2.2434 - accuracy: 0.4160
Epoch 7/10
1563/1563 [==============================] - 5s 3ms/step - loss: 2.1194 - accuracy: 0.4428
Epoch 8/10
1563/1563 [==============================] - 5s 3ms/step - loss: 2.0202 - accuracy: 0.4693
Epoch 9/10
1563/1563 [==============================] - 5s 3ms/step - loss: 1.9159 - accuracy: 0.4907
Epoch 10/10
1563/1563 [==============================] - 5s 3ms/step - loss: 1.834

[2.727922201156616, 0.35519999265670776]

> 일단 broadcasting 되어서 계산은 되는 것 같은데,  
> 학습이 되고 accuracy가 올라간다는 것은 결국 나머지 900개의 노드에서 거의 0에 가까운 의미없는 값들이 출력되기 때문에 무시되는 셈이 아닌가 싶다.

## Tensorflow2 API로 모델 작성 및 학습하기: CIFAR-100 (2) Functional API 활용
- 데이터는 위에서 불러왔으므로 건너 뛴다.

In [83]:
# Functional API를 활용한 Model을 구성해주세요.
"""
Spec:
0. (32X32X3) 차원으로 정의된 Input
1. 16개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
2. pool_size가 2인 MaxPool 레이어
3. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
4. pool_size가 2인 MaxPool 레이어
5. 256개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
6. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)
"""

# 여기에 모델을 구성해주세요

# --- Input
inputs = keras.layers.Input(shape=(32, 32, 3))

# --- Conv2D
conv2d = keras.layers.Conv2D(16, kernel_size=3, activation='relu')
x = conv2d(inputs)

# --- MaxPool
maxpool2d = keras.layers.MaxPool2D((2,2))
x = maxpool2d(x)

# --- Conv2D_1
conv2d_1 = keras.layers.Conv2D(32, kernel_size=3, activation='relu')
x = conv2d_1(x)

# --- MaxPool
maxpool2d_1 = keras.layers.MaxPool2D((2,2))
x = maxpool2d_1(x)

# --- Flatten
flatten = keras.layers.Flatten()
x = flatten(x)

# --- Dense
dense = keras.layers.Dense(256, activation='relu')
x = dense(x)

# --- Dense_1
dense_1 = keras.layers.Dense(100, activation='softmax')
outputs = dense_1(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [84]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 5s 3ms/step - loss: 4.0295 - accuracy: 0.0940
Epoch 2/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.9991 - accuracy: 0.2663
Epoch 3/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.6588 - accuracy: 0.3350
Epoch 4/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.4387 - accuracy: 0.3787
Epoch 5/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.2734 - accuracy: 0.4081
313/313 - 1s - loss: 2.6318 - accuracy: 0.3426


[2.6317834854125977, 0.3425999879837036]

## Tensorflow2 API로 모델 작성 및 학습하기: CIFAR-100 (3) Subclassing 활용

In [88]:
# Subclassing을 활용한 Model을 구성해주세요.
"""
Spec:
0. keras.Model 을 상속받았으며, __init__()와 call() 메서드를 가진 모델 클래스
1. 16개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
2. pool_size가 2인 MaxPool 레이어
3. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
4. pool_size가 2인 MaxPool 레이어
5. 256개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
6. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)
7. call의 입력값이 모델의 Input, call의 리턴값이 모델의 Output
"""

# 여기에 모델을 구성해주세요

class MyModel(keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv2d = keras.layers.Conv2D(16, 3, activation='relu')
        self.maxp = keras.layers.MaxPool2D((2,2))
        self.conv2d_1 = keras.layers.Conv2D(32, 3, activation='relu')
        self.maxp_1 = keras.layers.MaxPool2D(2,2)
        self.flatten = keras.layers.Flatten()
        self.dense = keras.layers.Dense(256, activation='relu')
        self.dense_1 = keras.layers.Dense(100, activation='softmax')
    
    def call(self, inputs):
        x = self.conv2d(inputs)
        x = self.maxp(x)
        x = self.conv2d_1(x)
        x = self.maxp_1(x)
        x = self.flatten(x)
        x = self.dense(x)
        outputs = self.dense_1(x)
        
        return outputs

In [89]:
model = MyModel()
sgd = keras.optimizers.SGD(momentum=0.2, nesterov=True)
model.compile(optimizer=sgd,
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 5s 3ms/step - loss: 4.5576 - accuracy: 0.0215
Epoch 2/5
1563/1563 [==============================] - 5s 3ms/step - loss: 4.0788 - accuracy: 0.0773
Epoch 3/5
1563/1563 [==============================] - 5s 3ms/step - loss: 3.7740 - accuracy: 0.1307
Epoch 4/5
1563/1563 [==============================] - 5s 3ms/step - loss: 3.5970 - accuracy: 0.1640
Epoch 5/5
1563/1563 [==============================] - 5s 3ms/step - loss: 3.4008 - accuracy: 0.1981
313/313 - 1s - loss: 3.3886 - accuracy: 0.2024


[3.3886260986328125, 0.20239999890327454]

In [90]:
model = MyModel()
sgd = keras.optimizers.SGD(momentum=0.9, nesterov=True)
model.compile(optimizer=sgd,
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 5s 3ms/step - loss: 4.2382 - accuracy: 0.0579
Epoch 2/5
1563/1563 [==============================] - 5s 3ms/step - loss: 3.2162 - accuracy: 0.2184
Epoch 3/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.7883 - accuracy: 0.3054
Epoch 4/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.5138 - accuracy: 0.3575
Epoch 5/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.2861 - accuracy: 0.4091
313/313 - 1s - loss: 2.7217 - accuracy: 0.3271


[2.721714496612549, 0.32710000872612]

## GradientTape의 활용

### Automatic differentiation - GradientTape

In [ ]:
import tensorflow as tf
from tensorflow import keras

# 데이터 구성부분
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

# 모델 구성부분
class CustomModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(16, 3, activation='relu')
        self.maxpool1 = keras.layers.MaxPool2D((2,2))
        self.conv2 = keras.layers.Conv2D(32, 3, activation='relu')
        self.maxpool2 = keras.layers.MaxPool2D((2,2))
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(256, activation='relu')
        self.fc2 = keras.layers.Dense(100, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)

        return x

model = CustomModel()

> 모델 설계에는 변함 없으나, compile과 fit 부분을 다르게 해볼 것

In [91]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# tf.GradientTape()를 활용한 train_step
def train_step(features, labels):
    with tf.GradientTape() as tape:
        predictions = model(features)
        loss = loss_func(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

> `tape.gradient()`로 매 step마다 발생하는 gradient를 추출하고,  
> `model.trainable_variables`를 통해 업데이트해야 할 파라미터를 주고 받고  
> `optimizer.apply_gradients()`를 통해 업데이트한다.

In [97]:
import time
def train_model(batch_size=32):
    start = time.time()
    for epoch in range(5):
        x_batch = []
        y_batch = []
        for step, (x, y) in enumerate(zip(x_train, y_train)):
            x_batch.append(x)
            y_batch.append(y)
            if step % batch_size == batch_size-1:
                loss = train_step(np.array(x_batch, dtype=np.float32), np.array(y_batch, dtype=np.float32))
                x_batch = []
                y_batch = []
        print('Epoch %d: last batch loss = %.4f' % (epoch, float(loss)))
    print("It took {} seconds".format(time.time() - start))

train_model()

Epoch 0: last batch loss = 2.6041
Epoch 1: last batch loss = 1.9978
Epoch 2: last batch loss = 1.7482
Epoch 3: last batch loss = 1.5230
Epoch 4: last batch loss = 1.2091
It took 94.82491636276245 seconds


In [98]:
# evaluation
prediction = model.predict(x_test, batch_size=x_test.shape[0], verbose=1)
temp = sum(np.squeeze(y_test) == np.argmax(prediction, axis=1))
temp/len(y_test)  # Accuracy

1/1 [==============================] - 6s 6s/step


0.319

> gradients를 사용 안해도 되는 evaluation 단계는 그냥 `model.predict()` 메소드 사용

-------------------------------------------------------------
### GradientTape 관련 연습

In [104]:
model.trainable_variables

[<tf.Variable 'my_model_11/conv2d_38/kernel:0' shape=(3, 3, 3, 16) dtype=float32, numpy=
 array([[[[ 1.56246386e-02, -6.42082334e-01,  2.73936719e-01,
            3.96442622e-01, -1.07740767e-01, -5.81818461e-01,
           -1.58907518e-01,  2.01875895e-01,  6.01437548e-03,
            5.67011118e-01, -1.92512676e-01, -4.17991787e-01,
            5.58841944e-01,  1.33760393e-01,  6.14034772e-01,
           -2.35914677e-01],
          [-3.31133723e-01,  1.18974410e-01, -2.76962012e-01,
            4.48547453e-01,  5.04576527e-02, -1.18975437e+00,
            6.84822440e-01, -1.00665577e-01,  1.31117716e-01,
           -2.97998726e-01, -1.22719444e-01,  1.28715605e-01,
           -3.91524702e-01,  8.70915013e-04,  6.09937429e-01,
           -3.30402702e-01],
          [ 2.48610839e-01, -5.01197018e-02,  2.56127179e-01,
            3.45690966e-01, -5.91540597e-02, -8.35236430e-01,
           -1.34304777e-01, -8.68157074e-02, -1.54668421e-01,
           -1.13675542e-01,  9.92290825e-02, -3

In [113]:
dir(tf.GradientTape())

['__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_created_eagerly',
 '_persistent',
 '_pop_tape',
 '_push_tape',
 '_recording',
 '_tape',
 '_tf_api_names',
 '_tf_api_names_v1',
 '_watch_accessed_variables',
 '_watched_variables',
 'batch_jacobian',
 'gradient',
 'jacobian',
 'reset',
 'stop_recording',
 'watch',
 'watched_variables']

In [117]:
with tf.GradientTape() as tape:
    anyth = tape.reset()
    b = 2
anyth

In [118]:
b

2

In [119]:
type(anyth)

NoneType

> - with 함수 안에서 할당한 변수가 밖에서도 계속 유지되는지(global scope로 되는지) 확인하고 싶었다.  
> - 그렇게 되는 걸 확인할 수 있다.  
> - 여기서 anyth는 Nonetype인데, 그래서인지 output이 없다.